In [13]:
# Demo test scene: Sidewalk near JCC building at Tufts University 
# Behind baseball field 
# Structure-from-Motion solution provided by COLMAP 
# Colmap generated: images, 3D points, cameras

import open3d as o3d
import numpy as np
import plotly.graph_objects as go 
from scipy.spatial.transform import Rotation as R
from colmapParsingUtils import *
import copy

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt

import tensorflow as tf
import cv2

%load_ext autoreload
%autoreload 2
%autosave 180

# # Limit GPU memory, if running notebook on GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:
#     try:
#         memlim = 2*1024
#         tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
#     except RuntimeError as e:
#         print(e)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [14]:
# Load images, points, pose estimations from COLMAP 

images_colmap = read_images_text('/home/daniel-choate/ASAR/s2/COLMAP/TuftsWalkway/images.txt')
cameras = read_cameras_text('/home/daniel-choate/ASAR/s2/COLMAP/TuftsWalkway/cameras.txt')
pts3d = read_points3D_text('/home/daniel-choate/ASAR/s2/COLMAP/TuftsWalkway/points3D.txt')


In [15]:
# Extract 3d points from raw file 

pointdata_fn = "/home/daniel-choate/ASAR/s2/COLMAP/TuftsWalkway/points3D.txt"
num_pts = 75497 # Gerrard Hall

# points_data = []
scene_pts = np.zeros([num_pts, 3]) 
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:]  # Skip the first three lines
    
    for line in lines:
        elements = line.strip().split()
        
        row = [float(elem) for elem in list(elements)]

        scene_pts[idx,0] = row[1]
        scene_pts[idx,1] = row[2]
        scene_pts[idx,2] = row[3]
        idx += 1

# Filter RGB data into separate numpy array 
rgb_data = np.zeros([num_pts, 3])
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:] # Skip the first three lines 
    for line in lines:
        elements = line.strip().split()
        row = [float(elem) for elem in list(elements)]

        rgb_data[idx,0] = row[4]
        rgb_data[idx,1] = row[5]
        rgb_data[idx,2] = row[6]
        idx += 1

# SCALE COLORS
rgb_data = rgb_data/255
# print(scene_pts[:10])

In [23]:
# Convert COLMAP poses (xyz,quats) to rotm

poses = np.zeros([len(images_colmap),4,4])
images = np.zeros([len(poses),250,250,3])


skip_indices = {56, 58}  # Use a set to store the indices to skip

#loop through <images_from_colmap> to get 3D poses of cameras at each timestamp
# print(len(images_from_colmap))
for n in range(len(images_colmap)):
    # if n in skip_indices:
        # continue


    # Pull quaternion and translation vector
    qvec = images_colmap[n+1].qvec #raw
    # print(qvec)
    # print(images_colmap[n+1].tvec)
    tvec = images_colmap[n+1].tvec[:,None]
    # print(tvec)
    
    t = tvec.reshape([3,1])
    # print(tvec)
    Rotmat = qvec2rotmat(-qvec)
    
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    m = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(m)

    # NERF SPECIFIC ROTATIONS 
    # c2w[0:3,2] *= -1 # flip the y and z axis
    # c2w[0:3,1] *= -1
    # c2w = c2w[[1,0,2,3],:]
    # c2w[2,:] *= -1 # flip whole world upside down
    


    poses[n] = c2w
    # poses[n] = m
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#GET REST OF PARAMS NEEDED FOR tinyNeRF format~~~~~~~~~~~~~~~~~~~~~~~~~~~~    

#fix order of colors
images[:,:,:,0], images[:,:,:,1] = images[:,:,:,1], images[:,:,:,0]

H,W = images.shape[1:3]
# print(H,W)
testimg, testpose = images[55], poses[55]

focal = cameras[1].params[0] #test- see if same focal length can be shared across all images
focal = focal/12
print(focal)
# print(poses)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

250.99782250100847


In [24]:
# np.save('poses_tuftsww.npy', poses)

# Plotting 3D point Cloud 

In [25]:
# Plotting 3D points
# BY ITSELF
# Data generated by COLMAP

vis = o3d.visualization.Visualizer()
vis.create_window()

scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

vis.add_geometry(scene_cloud)

# render_option = vis.get_render_option()

# render_option.background_color = [1, 1, 1] #[0.678, 0.847, 0.902] # Light blue

vis.run()

vis.destroy_window()


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 70
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 70
pci id for fd 70: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

# Plotting 3D coord frames

In [26]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)

# loop through poses 
for pose in poses:
    homog_t = pose
    axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    

vis.add_geometry(axes)
# vis.add_geometry(scene_cloud)

# Run the visualizer
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 70
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 70
pci id for fd 70: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 

# Coord frames and point cloud

In [27]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

# loop through poses 
for pose in poses:
    homog_t = pose
    axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    

vis.add_geometry(axes)
vis.add_geometry(scene_cloud)

# Run the visualizer
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 70
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 70
pci id for fd 70: 8086:a7a0, driver iris
MESA-LOADER: dlopen(/usr/lib/x86_64-linux-gnu/dri/iris_dri.so)
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using 